In [1]:
!pip install scanpy
import os
import numpy as np
import pandas as pd
import scanpy as sc
import time
from scipy.stats import pearsonr
#
#more 10x datasets 
#https://support.10xgenomics.com/single-cell-multiome-atac-gex/datasets/


#switch to keras for custom ae
#https://blog.keras.io/building-autoencoders-in-keras.html
#merging nerual networks
#https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



In [2]:
from google.colab import drive
drive.mount('/content/drive')
#make a shortcut to emily's shared drive folder in your drive so you can access the data at
import os
os.listdir('/content/drive/My Drive/methyl_impute')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Peek_data.ipynb',
 'Project Proposal Brainstorming.gdoc',
 'adult_cortex_CG-CH_OLDCOPY.h5ad',
 'adult_cortex_CG_CH.h5ad',
 'tf_multiclass_prediction.ipynb',
 'classifier_results.gslides',
 'ch_isImputed.parq',
 'cg_isImputed.parq',
 'PredictionModel.ipynb',
 'vanilla tuning',
 'VAE_impute (variational+loss).ipynb',
 'final_imputation_mask.parq',
 'VAE_impute (vanilla) (custom dropout+loss).ipynb',
 'VAE_tune',
 'VAE_impute (vanilla+tuned).ipynb',
 'VAE_impute (vanilla) (custom dropout).ipynb',
 'Presentation.gslides',
 'VAE_impute (variational).ipynb',
 'Project_Report.gdoc',
 'VAE_impute (variational+tuned).ipynb',
 'final_imputation_mask_2.parq',
 'final_imputation_mask_2.csv',
 'VAE_impute (variational) (custom dropout+custom loss).ipynb',
 'test_predictions_top_genes.pickle',
 'results.gdoc',
 'test_predictions_top_genes_plus_25.pickle',
 'test_predictions_top_genes_plus_50.pickle',
 'test_predictions_top_genes_plus_75.pickle']

In [3]:
adata=sc.read_h5ad('/content/drive/My Drive/methyl_impute/adult_cortex_CG_CH.h5ad')
adata

AnnData object with n_obs × n_vars = 11945 × 44772
    obs: 'sample', 'L1', 'L2', 'L3', 'true_batch', 'age', 'age_groups', 'leiden'
    var: 'batch'
    uns: 'L2_colors', 'L3_colors', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
#collect top marker genes

sc.tl.rank_genes_groups(adata, 'L3',n_genes=20)
# sc.pl.rank_genes_groups(adata, sharey=False)
markers=[]
for i in range(27):
    for j in range(20):
        if adata.uns['rank_genes_groups']['names'][j][i] not in markers:
            markers.append(adata.uns['rank_genes_groups']['names'][j][i])
"""adata2=adata[:,markers]
sc.pp.scale(adata2)
sc.pl.heatmap(adata2, markers, groupby='L3',vmax=1,vmin=-1)"""

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/usr/local/lib/python3.7/dist-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()

"adata2=adata[:,markers]\nsc.pp.scale(adata2)\nsc.pl.heatmap(adata2, markers, groupby='L3',vmax=1,vmin=-1)"

In [6]:
import random

print(random.random())

0.1374989565502055


In [7]:
for item in adata.var.index.values:
    if item not in markers:
        if random.random() <= 0.75:
            markers.append(item)

print(len(markers))        

33793


In [16]:
markers = adata.var.index.values
print(len(markers))

44772


In [8]:
adata=adata[:,markers]
adata

View of AnnData object with n_obs × n_vars = 11945 × 33793
    obs: 'sample', 'L1', 'L2', 'L3', 'true_batch', 'age', 'age_groups', 'leiden'
    var: 'batch'
    uns: 'L2_colors', 'L3_colors', 'leiden', 'neighbors', 'pca', 'umap', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [4]:
#data matrix, X
"""X=np.array(adata.X)
print(X.shape)"""

yl=adata.obs['L3']
print(yl.shape)

(11945,)


In [5]:
final_mask = pd.read_parquet('/content/drive/My Drive/methyl_impute/final_imputation_mask.parq')

In [11]:
final_mask = final_mask[adata.to_df().columns.values].values

In [6]:
final_mask.shape

(11945, 44772)

In [7]:
class DropoutWithInference(layers.Layer):
    def __init__(self, rate, **kwargs):
        super(DropoutWithInference, self).__init__(**kwargs)
        self.rate = rate

    def _mask(self, args):
        methyl, boolean_imput = args
        methyl *= boolean_imput
        return methyl

    def call(self, inputs, training=None):
        methyl, boolean_impute = inputs
        if training:
            return layers.Dropout(rate=self.rate)(methyl)
        else:
            return layers.Lambda(self._mask)(inputs)

In [8]:
class VAE:
  def __init__(self, input_shape,batch_size=256,optimizer=keras.optimizers.Adam(learning_rate=.001),epochs=100,
               recon_loss_function=tf.keras.losses.MeanSquaredError(),callback=tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)):
    self.mu = None
    self.sigma = None

    self.input_shape=input_shape
    self.batch_size=batch_size
    self.n_epochs=epochs
    self.optimizer=optimizer
    self.recon_loss_function=recon_loss_function
    self.callback=callback
    self.encoder=self.generate_encoder()
    self.encoder.compile(optimizer=optimizer)
    self.sampler=self.perform_sampling()
    self.decoder=self.generate_decoder()
    self.decoder.compile(optimizer=optimizer)

  def vae_loss(self, y_true, y_pred, mask):
    recon = self.custom_recon_loss(y_true, y_pred, mask)
    kl = K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))
    kl = tf.cast(kl, tf.float64)
    return recon + kl/200

  def sample_z(self, args):
    vae_mu, vae_sigma = args
    eps = K.random_normal(shape=(K.shape(vae_mu)[0], K.shape(vae_mu)[1]), mean=0., stddev=1., seed=42)
    return vae_mu + K.exp(vae_sigma/2)*eps

  def kl_loss(self):
    return K.mean(0.5*K.sum(K.exp(self.sigma) + K.square(self.mu) - 1. - self.sigma, axis=1))

  def custom_recon_loss(self, label_true, prediction, mask):
    squared_error = tf.math.square(tf.math.subtract(tf.cast(label_true, tf.float64), tf.cast(prediction, tf.float64)))
    masked_error = tf.math.multiply(squared_error, mask)
    return tf.math.reduce_mean(masked_error)

  def generate_encoder(self):
    mC_input=keras.Input(shape=(self.input_shape,),name='mC_input')
    imputed_boolean_input = keras.Input(shape=(self.input_shape,), name='imput_mask_input')
    input_dropout = DropoutWithInference(0.25, name='custom_dropout')([mC_input, imputed_boolean_input])
    dense1=layers.Dense(128,activation='relu', name='dense1')(input_dropout)
    # dropout1=layers.Dropout(0.25, name='dropout1')(dense1)
    dense2=layers.Dense(64,activation='relu', name='dense2')(dense1)
    # dropout2=layers.Dropout(0.25, name='dropout2')(dense2)
    vae_mu=layers.Dense(16,activation='linear')(dense2)
    vae_sigma=layers.Dense(16,activation='linear')(dense2)

    return Model([mC_input, imputed_boolean_input], (vae_mu, vae_sigma), name='encoder')

  def perform_sampling(self):
    vae_mu = keras.Input(shape=(16,))
    vae_sigma = keras.Input(shape=(16,))
    out=layers.Lambda(self.sample_z)([vae_mu, vae_sigma])
    return Model([vae_mu, vae_sigma], out, name='sampler')

  def generate_decoder(self):
    input_latent = keras.Input(shape=(16,))
    out_pred=layers.Dense(self.input_shape,activation='linear',name='out_pred')(input_latent)
    return Model(input_latent, out_pred, name='decoder')

  def train_vae(self,train_dataset_vals,train_dataset_mask,test_dataset_vals,test_dataset_mask):
    epochs = self.n_epochs
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        epoch_train_losses = []

        # Iterate over the batches of the dataset.
        for (step, (x_batch_train_vals)), (step, (x_batch_train_mask)) in zip(enumerate(train_dataset_vals), enumerate(train_dataset_mask)):
            with tf.GradientTape() as encoder_tape, tf.GradientTape() as decoder_tape:
                self.mu, self.sigma = self.encoder([x_batch_train_vals, x_batch_train_mask], training=True)
                latent = self.sampler([self.mu, self.sigma])
                logits=self.decoder(latent, training=True)
                loss_value = self.vae_loss(x_batch_train_vals, logits, x_batch_train_mask)

            encoder_grads = encoder_tape.gradient(loss_value, self.encoder.trainable_weights)
            decoder_grads = decoder_tape.gradient(loss_value, self.decoder.trainable_weights)
            self.optimizer.apply_gradients(zip(encoder_grads, self.encoder.trainable_weights))
            self.optimizer.apply_gradients(zip(decoder_grads, self.decoder.trainable_weights))

            # Update training metric.
            epoch_train_losses.append(self.vae_loss(x_batch_train_vals, logits, x_batch_train_mask))

            # Log every 31 batches.
            if step % 31 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )

        # Display metrics at the end of each epoch.
        train_acc = sum(epoch_train_losses)/len(epoch_train_losses)
        print("Reconstruction Training acc over epoch: %.4f" % (float(train_acc),))
        print("KL Divergence: %.10f" % (float(self.kl_loss())))

        epoch_val_losses = []
        # Run a validation loop at the end of each epoch.
        for (x_batch_test_vals, x_batch_test_mask) in zip(test_dataset_vals, test_dataset_mask):
            self.mu, self.sigma = self.encoder([x_batch_test_vals, x_batch_test_mask], training=False)
            latent = self.sampler([self.mu, self.sigma])
            test_logits = self.decoder(latent, training=False)
            epoch_val_losses.append(self.vae_loss(x_batch_test_vals, test_logits, x_batch_test_mask))
        print("Validation loss: %.4f" % (float(sum(epoch_val_losses)/len(epoch_val_losses)),))

  def predict(self,X_vals,X_mask):
    self.mu, self.sigma = self.encoder.predict([X_vals, X_mask])
    latent = self.sampler([self.mu, self.sigma])
    return self.decoder.predict(latent)

  def model_summary(self):
    print(self.model.summary())

  def visualize_model(self):
    #dunno how to make this show from the function lol
    keras.utils.plot_model(self.model, show_shapes=True, show_layer_names=True)

  def correlation_accuracy(self, real_data_vals, real_data_mask):
    reconstructed_data = self.predict(real_data_vals, real_data_mask)
    corrs = 0
    for i in range(len(reconstructed_data)):
        corrs += pearsonr(reconstructed_data[i], real_data_vals[i])[0]

    print(corrs/len(reconstructed_data))

  def mse_error(self, real_data_vals, real_data_mask):
    reconstructed_data = self.predict(real_data_vals, real_data_mask)
    mse = 0
    for i in range(len(reconstructed_data)):
        mse += mean_squared_error(reconstructed_data[i], real_data_vals[i])

    print(self.custom_recon_loss(real_data_vals, reconstructed_data, real_data_mask))
    print(mse/len(reconstructed_data))

In [9]:
temp_input = np.concatenate([adata.X, final_mask], axis=1)

X_train, X_test, y_train, y_test = train_test_split(temp_input, yl, test_size=0.33, random_state=42,shuffle=True)
X_train_vals = X_train[:, :adata.X.shape[1]]
X_train_mask = X_train[:, adata.X.shape[1]:]
X_test_vals = X_test[:, :adata.X.shape[1]]
X_test_mask = X_test[:, adata.X.shape[1]:]
print(X_train_vals.shape)
print(X_test_vals.shape)
print(X_train_mask.shape)
print(X_test_mask.shape)


(8003, 44772)
(3942, 44772)
(8003, 44772)
(3942, 44772)


In [10]:
print(y_test)

20210528-UMB5577-preAb-PFC-B8_O8        MGE ERBB4
20210224-5577-postAb-PFC-C08_F10              ODC
20210505-UMB6096-preAb-PFC-B03_F3       L1-3 CUX2
20210528-UMB5621-preAb-PFC-A1_A24             ODC
SRR9012214                                L4 RORB
                                         ...     
20210528-UMB5621-preAb-PFC-A2_O24           Astro
SRR9012607                                  Astro
SRR9012169                                  Astro
20210505-UMB5577-preAb-PFC-B09_F16    CT-L6 ASTN1
20210505-UMB5577-preAb-PFC-A09_O5      CGE SPOCK3
Name: L3, Length: 3942, dtype: category
Categories (27, object): ['Astro', 'CGE SOX13', 'CGE SPOCK3', 'CGE TOX', ..., 'ODC', 'OPC', 'PC',
                          'VLMC']


In [11]:
temp_input = None

In [12]:
final_mask = None

In [17]:
adata = None

In [13]:
X_train = None

In [14]:
X_test = None

In [18]:
vae=VAE(X_train_vals.shape[1])

In [19]:
batch_size=256

train_dataset_vals=tf.data.Dataset.from_tensor_slices(X_train_vals)
train_dataset_vals = train_dataset_vals.shuffle(buffer_size=1024, seed=42).batch(batch_size)

train_dataset_mask=tf.data.Dataset.from_tensor_slices(X_train_mask)
train_dataset_mask = train_dataset_mask.shuffle(buffer_size=1024, seed=42).batch(batch_size)

test_dataset_vals = tf.data.Dataset.from_tensor_slices(X_test_vals)
test_dataset_vals = test_dataset_vals.batch(batch_size)

test_dataset_mask = tf.data.Dataset.from_tensor_slices(X_test_mask)
test_dataset_mask = test_dataset_mask.batch(batch_size)

In [30]:
train_dataset_vals = None
train_dataset_mask = None
test_dataset_vals = None
test_dataset_mask = None

In [20]:
X_train_vals = None
X_train_mask = None
# X_test_vals = None
# X_test_mask = None

In [21]:
vae.train_vae(train_dataset_vals,train_dataset_mask,test_dataset_vals,test_dataset_mask)


Start of epoch 0
Training loss (for one batch) at step 0: 0.1841
Training loss (for one batch) at step 31: 0.1363
Reconstruction Training acc over epoch: 0.3440
KL Divergence: 5.5703349113
Validation loss: 0.1363

Start of epoch 1
Training loss (for one batch) at step 0: 0.1356
Training loss (for one batch) at step 31: 0.0795
Reconstruction Training acc over epoch: 0.1039
KL Divergence: 6.7262020111
Validation loss: 0.0746

Start of epoch 2
Training loss (for one batch) at step 0: 0.0748
Training loss (for one batch) at step 31: 0.0503
Reconstruction Training acc over epoch: 0.0603
KL Divergence: 5.0526971817
Validation loss: 0.0500

Start of epoch 3
Training loss (for one batch) at step 0: 0.0490
Training loss (for one batch) at step 31: 0.0446
Reconstruction Training acc over epoch: 0.0454
KL Divergence: 4.6226949692
Validation loss: 0.0417

Start of epoch 4
Training loss (for one batch) at step 0: 0.0410
Training loss (for one batch) at step 31: 0.0378
Reconstruction Training acc o

In [22]:
predictions = vae.predict(X_test_vals, X_test_mask)

In [29]:
predictions = None

In [23]:
predictions.shape

(3942, 44772)

In [24]:
len(markers)

44772

In [26]:
temp = pd.DataFrame(predictions, columns=markers, index=y_test.index.values)
temp['L3'] = y_test.values
print(temp.shape)

temp.to_pickle('/content/drive/My Drive/methyl_impute/test_predictions_all_genes.pickle')


(3942, 44773)


In [28]:
temp = None

In [31]:
# vae.correlation_accuracy(X_train)
# vae.correlation_accuracy(X_test_vals, X_test_mask)
# vae.mse_error(X_train)
vae.mse_error(X_test_vals, X_test_mask)

tf.Tensor(0.006814523288417324, shape=(), dtype=float64)
0.010413017228384632
